In [240]:
import torch
from torchsummary import summary
import numpy as np
import json
import math

from typing import List, Dict, Tuple
from tqdm import tqdm

In [4]:
data = json.load(open('processed_spec_data_sample.json', 'r'))

In [77]:
class Spec2FragsDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        data : List[Dict[str, List[int]]]
    ) -> None:
        
        self.spectrums = []
        self.frags = []
        
        for cur in data:
            self.spectrums.append(
                torch.cat([
                    torch.tensor([SPECS_BOS_TOKEN], dtype=torch.int64),
                    self._get_labels_from_spectrum(
                        torch.tensor(
                            cur['spectrum'],
                            dtype=torch.int64
                        )
                    ),
                    torch.tensor([SPECS_EOS_TOKEN], dtype=torch.int64)
                ])
            )
            self.frags.append(
                torch.cat([
                    torch.tensor([FRAGS_BOS_TOKEN], dtype=torch.int64),
                    torch.tensor(
                        cur['frags'],
                        dtype=torch.int64
                    ),
                    torch.tensor([FRAGS_EOS_TOKEN], dtype=torch.int64)
                ])
            )

    def _get_labels_from_spectrum(
        self,
        spectrum : torch.Tensor
    ) -> torch.Tensor:
        res = []
        for idx in (spectrum > 0).nonzero().flatten():
            res.extend([idx] * spectrum[idx])
        return torch.stack(res)

    def __len__(
        self
    ) -> int:
        return len(self.spectrums)

    def __getitem__(
        self,
        idx
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.spectrums[idx], self.frags[idx]

In [78]:
dataset = Spec2FragsDataset(data=data)

In [79]:
dataset[0]

(tensor([300,  68,  92, 169, 174, 176, 177, 190, 199, 301]),
 tensor([4000,  866,   13,   10,  855,   11, 4001]))

In [80]:
train_idxs = np.random.choice(np.arange(len(dataset)), int(0.8*len(dataset)), replace=False)
val_mask = [True for _ in range(len(dataset))]
for cur in train_idxs:
    val_mask[cur] = False
val_idxs = np.arange(len(dataset))[val_mask]

In [81]:
assert val_idxs.shape[0] + train_idxs.shape[0] == len(dataset)
assert np.intersect1d(val_idxs, train_idxs).shape[0] == 0

In [82]:
train_set = torch.utils.data.Subset(dataset, train_idxs)
val_set = torch.utils.data.Subset(dataset, val_idxs)

In [293]:
specs_vocab_size = 300

SPECS_BOS_TOKEN = specs_vocab_size
SPECS_EOS_TOKEN = specs_vocab_size + 1
SPECS_PAD_TOKEN = specs_vocab_size + 2

# Correct
frags_vocab_size = 5500
FRAGS_BOS_TOKEN = frags_vocab_size
FRAGS_EOS_TOKEN = frags_vocab_size + 1
FRAGS_PAD_TOKEN = frags_vocab_size + 2

In [279]:
def collate_fn(
    data : List[Tuple[torch.Tensor, torch.Tensor]]
):
    specs = []
    frags = []
    for cur in data:
        specs.append(cur[0])
        frags.append(cur[1])
    return (
        torch.nn.utils.rnn.pad_sequence(
            specs, 
            batch_first=True, 
            padding_value=SPECS_PAD_TOKEN
        ),
        torch.nn.utils.rnn.pad_sequence(
            frags, 
            batch_first=True, 
            padding_value=FRAGS_PAD_TOKEN
        )
    )

In [280]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [281]:
next(iter(train_loader))

(tensor([[300, 110, 115, 119, 155, 167, 183, 186, 188, 202, 301, 302],
         [300,  70,  77,  86,  92, 117, 134, 202, 213, 220, 224, 301],
         [300, 105, 176, 179, 179, 186, 219, 301, 302, 302, 302, 302],
         [300,  59,  81, 166, 172, 176, 184, 185, 192, 223, 236, 301]]),
 tensor([[4000, 1585,  635, 1905,   19,   17,  791,  660,    5, 1343, 1260, 1584,
          1236, 4001],
         [4000, 1170,   90,  260, 1596,  612,  262,  756,   95, 3067,  265,  350,
          4001, 5502],
         [4000, 3423,   13, 3424,  926, 4919,  107,  102, 4001, 5502, 5502, 5502,
          5502, 5502],
         [4000,  131, 1271, 1846, 2134,  644,  619,    6,   12,  313,  133,  321,
          4001, 5502]]))

In [282]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = torch.nn.Linear(d_model, d_model)
        self.W_k = torch.nn.Linear(d_model, d_model)
        self.W_v = torch.nn.Linear(d_model, d_model)
        self.W_o = torch.nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

class PositionWiseFeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(d_model, d_ff)
        self.fc2 = torch.nn.Linear(d_ff, d_model)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)
        self.norm3 = torch.nn.LayerNorm(d_model)
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

class Transformer(torch.nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = torch.nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = torch.nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = torch.nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = torch.nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = torch.nn.Linear(d_model, tgt_vocab_size)
        self.dropout = torch.nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != SPECS_PAD_TOKEN).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != FRAGS_PAD_TOKEN).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output


In [303]:
model = Transformer(
    src_vocab_size=specs_vocab_size + 3,
    tgt_vocab_size=frags_vocab_size + 3,
    d_model=16,
    num_heads=4,
    num_layers=2,
    d_ff=32,
    max_seq_length=100,
    dropout=0.2
)

In [304]:
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
NUM_EPOCHS = 100

In [311]:
train_loss = []
val_loss = []

for epoch in tqdm(range(NUM_EPOCHS)):
    cur_train_loss = []
    cur_val_loss = []
    
    model.train()
    for idx, (spectrum_train, frags_train) in enumerate(train_loader):
        preds = model(
            src=spectrum_train,
            tgt=frags_train[:, :-1]
        )
        loss = criterion(
            preds.view(-1, frags_vocab_size+3), 
            frags_train.contiguous()[:, 1:].reshape(-1)
        )

        opt.zero_grad()
        loss.backward()
        opt.step()

        cur_train_loss.append(loss.item())

        if np.random.rand() > 0.9:
            print(f"{idx} {cur_train_loss[-1]}")

    model.eval()
    with torch.inference_mode():
        for idx, (spectrum_val, frags_val) in enumerate(val_loader):
            preds = model(
                src=spectrum_val,
                tgt=frags_val[:, :-1]
            )
            loss = criterion(
                preds.view(-1, frags_vocab_size+3), 
                frags_val.contiguous()[:, 1:].reshape(-1)
            )
    
            cur_val_loss.append(loss.item())

            if np.random.rand() > 0.9:
                print(f"{idx} {cur_val_loss[-1]}")

    train_loss.append(np.mean(cur_train_loss))
    val_loss.append(np.mean(cur_val_loss))

    if epoch % 1 == 0:
        print(f"Epoch {epoch}; Last train loss: {train_loss[-1]}; Last val loss: {val_loss[-1]}")

  0%|                                                                                                                                                                 | 0/100 [00:00<?, ?it/s]

1 2.7495367527008057
2 3.792525291442871
11 3.5020346641540527


  1%|█▌                                                                                                                                                       | 1/100 [00:00<01:04,  1.53it/s]

Epoch 0; Last train loss: 3.542299520969391; Last val loss: 4.964497804641724
3 2.7574265003204346
4 4.037358283996582
11 3.8039982318878174


  2%|███                                                                                                                                                      | 2/100 [00:01<01:03,  1.55it/s]

...

16 3.456929922103882
Epoch 3; Last train loss: 3.636236083507538; Last val loss: 4.94342303276062


  5%|███████▋                                                                                                                                                 | 5/100 [00:03<00:57,  1.65it/s]

19 3.647644281387329
0 3.6085426807403564
2 6.758672714233398
Epoch 4; Last train loss: 3.6618675351142884; Last val loss: 4.9473975658416744
3 4.3535475730896


  5%|███████▋                                                                                                                                                 | 5/100 [00:03<01:04,  1.48it/s]


KeyboardInterrupt: 